In [1]:
class Sample:
    def __init__(self):
        self.data={}
        self.label=None
    def __setitem__(self,key,value):
        self.data[key]=value
    def __getitem__(self,key):
        return self.data[key]
    def __str__(self):
        return str(self.data)+'--label:'+str(self.label)
    def keys(self):
        return self.data.keys()
    def __hash__(self):
        return hash(str(self.data))
    def __cmp__(self,other):
        return cmp(str(self.data),str(other.data))

def label0(sample):
    return sample.label==0
def label1(sample):
    return sample.label==1

class Dataset:
    def __init__(self):
        self.data=[]
        self.ks=[]
    def __getitem__(self,key):
        return self.data[key]
    
    def append(self,sample):
        if(isinstance(sample,Sample)):
            self.data.append(sample)
        else:
            raise Exception('cannot append type:'+str(type(sample)))
        for k in sample.keys():
            if k not in self.ks:
                self.ks.append(k)
                
    def __str__(self):
        listOfSample=[]
        s_=''
        for k in self.ks:
            s_+=str(k)+'\t'
        listOfSample.append(s_+'label')
        
        for s in self.data:
            oneSample=[]
            for k in self.ks:
                s_ = ''
                if k in s.keys():
                    s_ = str(s[k])
                else:
                    s_ = 'NUL'
                oneSample.append(s_)
            oneSample.append(str(s.label))
            listOfSample.append('\t'.join(oneSample))
        return '\n'.join(listOfSample)+'\n'
    
    def split(self,preds):
        sets=[]
        for pred in preds:
            ds = Dataset()
            for s in self:
                if pred(s):
                    ds.append(s)
            sets.append(ds)
        return sets
    
    def canSplit(self):
        if len(set(self.data))>1:
            return True
        return False
    
    def vals(self,k):
        if k not in self.ks:
            return []
        vs=[]
        for s in self.data:
            vs.append(s[k])
        return vs
    def __len__(self):
        return len(self.data)
    def count(self,pred):
        c=0
        for s in self:
            if(pred(s)):
                c+=1
        return c
    
    def majorityVote(self):
        n0 = self.count(label0)
        n1 = self.count(label1)
        if n0<n1 : return 1
        else: return 0


In [2]:
class Filler:
    def fill(self,sample):
        pass

import random
class RandomFiller(Filler):
    def __init__(self):
        self.begin=0
        self.end=4
        self.nFeat=2
        self.nSamples=20
    def fillOne(self,sample):
        for i in range(self.nFeat):
            sample[i]=random.randint(self.begin,self.end)
        sample.label=random.randint(0,1)
    def fillDataset(self,dataset):
        for i in range(self.nSamples):
            s = Sample()
            self.fillOne(s)
            dataset.append(s)

In [3]:
a=Sample()
f=RandomFiller()
f.fillOne(a)
print(a)

{0: 1, 1: 4}--label:0


In [4]:
DS=Dataset()
f.fillDataset(DS)
print(DS)

0	1	label
2	4	1
4	3	0
0	3	1
0	4	0
1	3	1
2	3	0
0	3	1
2	1	1
3	4	1
2	0	0
1	1	1
0	2	1
4	3	1
2	0	1
4	2	0
3	4	0
2	2	1
2	4	0
0	0	1
4	2	0



In [5]:
class Model:
    def build(self,train):
        pass
    def predict(self,ob):
        pass

class DTNode:
    def __init__(self,name,parent):
        self.name = name
        self.parent = parent
        self.children=[]
        self.bPure=False
    def prepareSplit(dataset):
        # 1. which feature to split
        # 2. condition of each split : form a functor
        # 3. return feature and listOfFunctors
        pass
    def createBranch(dataset):
        # 1. Create some child nodes from some dataset,
        # 2. return true or false to determine the termination.
        pass

class DecisionTree(Model):
    def __init__(self):
        self.root = DTNode('root',None)
    def build(self,train):
        pass
    def predict(self,ob):
        pass

In [6]:
class Pred:
    pass

class EqualPred(Pred):
    def __init__(self,key,val):
        self.key=key
        self.val=val
    def __str__(self):
        return 'Feature'+str(self.key)+'=='+str(self.val)
    def __call__(self,sample):
        return sample[self.key]==self.val
    def __repr__(self): #just for display in array
        return self.__str__()

    
def EqualFunctors(dataset, key):
    vals = set(dataset.vals(key))
    functors=[]
    for v in vals:
        functors.append(EqualPred(key,v))
    return functors

import math
def calEntropy(dataset):
    
    n = len(dataset)
    nl0= dataset.count(label0)
    nl1= dataset.count(label1)
    if nl0==0 or nl1==0:
        return 0
    p0 = float(nl0)/n
    p1 = float(nl1)/n
    return -(p0 * math.log(p0,2)+p1*math.log(p1,2))
    
def InformationGain(dataset,key):
    preds = EqualFunctors(dataset,key)
    sets = dataset.split(preds)
    
    oldEntr = calEntropy(dataset)
    n = len(dataset)
    newEntr = 0
    for ds in sets:
        nds = len(ds)
        newEntr += calEntropy(ds) * nds/n
    
    return oldEntr-newEntr,key,preds,sets


InformationGain(DS,0)

(0.18338309822901333,
 0,
 [Feature0==0, Feature0==1, Feature0==2, Feature0==3, Feature0==4],
 [<__main__.Dataset instance at 0x104c6c248>,
  <__main__.Dataset instance at 0x104c6c170>,
  <__main__.Dataset instance at 0x104c75128>,
  <__main__.Dataset instance at 0x104c75a28>,
  <__main__.Dataset instance at 0x104c76170>])

In [7]:
class DTNode:
    def __init__(self,name,parent,ds):
        self.name = name
        self.parent = parent
        self.children={}
        self.label = ds.majorityVote()
        self.needSplit=False
        if ds.canSplit():
            nlabel = ds.count(lambda x:x.label==self.label)
            if nlabel != len(ds):
                self.needSplit = True
        self.isLeaf = not self.needSplit
        #if(self.isLeaf):
        #    print "label:"+ str(self.label)
        #    print ds
        
    def prepareSplit(self,dataset):
        res = []
        for k in dataset.ks:
            res.append(InformationGain(dataset,k))
        maxIGRes = max(res,key=lambda s:s[0])
        return maxIGRes
        
    def createBranch(self,dataset):
        _,key,preds,sets = self.prepareSplit(dataset)
        for p,s in zip(preds,sets):
            node = DTNode(str(p),self,s)
            if node.needSplit:
                node.createBranch(s)
            self.children[p] = node

class DecisionTree(Model):
    def __init__(self):
        self.root = None
    def build(self,train):
        self.root = DTNode('root',None,train)
        self.root.createBranch(train)
        print "model builded"
        
    def predict(self,ob):
        cur = self.root
        return self._predict(ob,cur)

    def _predict(self,ob,cur):
        if cur.isLeaf:
            return cur.label
        else:
            for pred in cur.children:
                if(pred(ob)):
                    cur = cur.children[pred]
                    break
            return self._predict(ob,cur)
        
DT = DecisionTree()
DT.build(DS)
DS.data.sort()
for s in DS:
    if DT.predict(s)!=s.label:
        print s,"failed"
    else:
        print s,"success"

model builded
{0: 0, 1: 0}--label:1 success
{0: 0, 1: 2}--label:1 success
{0: 0, 1: 3}--label:1 success
{0: 0, 1: 3}--label:1 success
{0: 0, 1: 4}--label:0 success
{0: 1, 1: 1}--label:1 success
{0: 1, 1: 3}--label:1 success
{0: 2, 1: 0}--label:0 success
{0: 2, 1: 0}--label:1 failed
{0: 2, 1: 1}--label:1 success
{0: 2, 1: 2}--label:1 success
{0: 2, 1: 3}--label:0 success
{0: 2, 1: 4}--label:1 failed
{0: 2, 1: 4}--label:0 success
{0: 3, 1: 4}--label:1 failed
{0: 3, 1: 4}--label:0 success
{0: 4, 1: 2}--label:0 success
{0: 4, 1: 2}--label:0 success
{0: 4, 1: 3}--label:0 success
{0: 4, 1: 3}--label:1 failed
